In [1]:
import importtestsmodule
import json
import kafka_client
import utils
import test_da
import test_jms
import test_an
import pprint

config_path = "../config.json"
cred_twitter_path = "../twitter.cred"
cred_reddit_path = "../reddit.cred"
pp=pprint.PrettyPrinter()

## Configuration and topics

### Config

`kafka_server` - set to `localhost:9094` (use `kafka:9092` if you run it inside a container)

### Topics

`sentiment` - sentiment analyser

### Credentials

Stored in a file `cred.json`. Some data acquirers require credentials.


In [6]:
pp.pprint(utils.load_config(cred_twitter_path))
pp.pprint(utils.load_config(cred_reddit_path))
pp.pprint(utils.load_config(config_path))

config loaded successfully
{'AccessToken': '1178262834595209216-2TTiiia1yOVd2UHHH07dBd7KDBN7pZ',
 'AccessTokenSecret': 'fhAw5o5IeTTc33mGtpFnRKGMpbEA6xcsnkIrFZBNZXq4c',
 'ApiKey': 'b0sXpwv0k4eJYmLaBZvSfNyxk',
 'ApiSecretKey': 'KK7aRtfR8XGFqaz2Ua1vDgYftQ01AqlUYXKkqrpWoVxR8FC0p7'}
config loaded successfully
{'AppId': 'Mx2Rp1J2roDMdg',
 'AppSecret': 'eDT3-0no1WHyTuBWTLoNDQNUqWA',
 'RefreshToken': '291925913345-DFbyOHX5f6zz-__Dqbr41jCOoPs'}
config loaded successfully
{'kafka': {'kafka_server': 'localhost:9094'},
 'topics': {'data_acquirer_update_topic': 'job_management.job_configuration.DataAcquirer_Twitter',
            'registration_channel_name': 'job_management.registration.request',
            'sentiment': 'job_management.component_data_input.DataAnalyser_topics'},
 'uris': {'jms_submit_uri': 'http://localhost:5100/api/job/submit'}}


# Tests

## Test data acquirer - data acquisition

**Prerequisities**:
- The data acquirer listents at a topic `config['topics']['dataacquirer_update_topic']`
- Credentials in case of a twitter data acquirer or reddit data acquirer.

**Steps**

- Creates new job
- Waits for one message
- Checks that the message is in the a correct post
  - Success if true, fails otherwise
- Stops the running job

In [7]:
print("Testing Twitter data acquirer")
test_da.start_test("../config.json",cred_twitter_path)

print("Testing Reddit data acquirer")
test_da.start_test("../config.json",cred_reddit_path)

Testing Twitter data acquirer
config loaded successfully
credentials loaded successfully
Create new job
Waits for acquired posts


Stop the job


KeyboardInterrupt: 

## Test job service registration and replay

**Prerequisites**:

- Jms must run (on `localhost:5100` in debug or `localhost:6009` in docker both defined in `config[uris][jms_submit_uri]`)

**Steps**:

- Registering new components
- Submit job config
- Assert received job configs for each component
  - Asserting data acquirer job config
  - Asserting analyser job config
- Assert replayed data acquirer config
  - Asserting data acquirer job config
  - Asserting analyser job config

In [11]:
test_jms.start_test("../config.json")

config loaded successfully


NoBrokersAvailable: NoBrokersAvailable

## Test analysers

**Prerequisites**:

- Analyser is running
- It ouptuts data to `job_management.component_data_analyzed_input.storage_db`

**Steps**:
 
- Produces a single post
- Checks analysis

In [2]:
test_an.start_test(config_path, "job_management.component_data_input.DataAnalyser_topics")

config loaded successfully
Produce post
Post produced - waiting for the analysis
{'postId': '0bb89a04-2f35-41bc-aa17-3d9dc159052d', 'jobId': 'e3a7ce8b-2d10-46ff-ba91-925a5a99a3e9', 'componentId': 'DataAnalyser_topics', 'results': {'topics': {'textListValue': ['model', 'type', 'topic', 'frequently', 'learning', 'semantic', 'processing', 'hide', 'text', 'structure', 'occur', 'collection', 'document', 'abstract', 'body', 'statistical', 'discovery', 'natural', 'tool', 'modeling', 'machine', 'mining']}}}
success
